<a href="https://colab.research.google.com/github/dudgus7477/datascience/blob/main/(9.1%EC%88%98%ED%96%89)(%ED%95%99%EB%B2%88%EC%9D%B4%EB%A6%84)%EC%87%BC%ED%95%91%20%EB%A6%AC%EB%B7%B0%20%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 네이버 쇼핑 리뷰 감성 분석
> naver_shopping.txt
- 언어: 한국어
- 출처: 네이버 쇼핑 (https://shopping.naver.com/)
- 수집 기간: 2020.06~2020.07
- 데이터 건수: 20만 건

- 네이버 쇼핑에서 제품별 후기를 별점과 함께 수집한 것입니다. 데이터는 탭으로 분리되어 있으며, 첫번째 필드에는 별점(1 ~ 5), 두번째 필드에는 텍스트가 위치합니다.

#1. 필요한 라이브러리 설치

In [1]:
# Colab에 Mecab 설치
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


#2. 데이터 로드하기

In [ ]:
df = pd.read_table('./naver_shopping.txt', encoding = 'UTF8')
df.head()

,star,review
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


In [ ]:
#전체 리뷰 개수 구하기 [빈칸1]


#3. 데이터 전처리
- 한글과 공백 제외한 나머지 제거

In [ ]:
#한글과 공백 제외한 나머지 제거후 데이터 확인 [빈칸2]


In [ ]:
#중복 제외한 리뷰의 개수는? [빈칸3]


In [ ]:
#중복 제거 [빈칸4]


In [ ]:
#널 값 유무를 확인해보자(True나 False중 하나 출력되야 함) [빈칸5]

- 별점이 4,5인 리뷰에는 레이블 1(긍정)을 별점이 1,2,3인 리뷰에는 레이블 0(부정)을 저장

In [ ]:
#실행해서 확인해보자
df['label'] = np.select([df.star > 3], [1], default=0)
df[:5]

- 훈련 데이터의 레이블 분포를 확인해보자.

In [ ]:
#레이블의 비율이 적절한가? 레이블 분포를 'bar' 그래프로 나타내보자! [빈칸6]


#4. 형태소로 토큰화

- 불용어 처리

In [ ]:
#불용어 지정하기
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인',
             '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

In [ ]:
# 불용어 제거 처리하기(mecab 사용하세요!)  [빈칸7]
mecab = Mecab()


#5. 리뷰 길이 분포 확인
- 긍정 리뷰에는 어떤 단어들이 많이 등장하고, 부정 리뷰에는 어떤 단어들이 등장하는지 빈도수 계산

In [ ]:
negative_words = np.hstack(df[df.label == 0]['tokenized'].values)
positive_words = np.hstack(df[df.label == 1]['tokenized'].values)

In [ ]:
#부정 단어 상위 50개 단어 출력 [빈칸8]


In [ ]:
#긍정 단어 상위 50개 단어 출력 [빈칸9]


- 길이 분포를 확인

In [ ]:
#실행해주세요
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10,5))
text_len = df[df['label']==1]['tokenized'].map(lambda x: len(x))
ax1.hist(text_len, color='red')
ax1.set_title('Positive Reviews')
ax1.set_xlabel('length of samples')
ax1.set_ylabel('number of samples')
print('긍정 리뷰의 평균 길이 :', np.mean(text_len))

text_len = df[df['label']==0]['tokenized'].map(lambda x: len(x))
ax2.hist(text_len, color='blue')
ax2.set_title('Negative Reviews')
fig.suptitle('Words in texts')
ax2.set_xlabel('length of samples')
ax2.set_ylabel('number of samples')
print('부정 리뷰의 평균 길이 :', np.mean(text_len))
plt.show()


# @ 왜 부정리뷰가 긍정리뷰보다 길게 작성될까?
> 이유를 2가지 생각해봅시다.
- 1.
- 2.

#6. 워드클라우드
- 긍정적인 키워드와 부정적인 키워드를 워드클라우드로 만들어 비교해보자!

In [ ]:
#한글 폰트 설치
! apt-get install fonts-nanum
font_path='/usr/share/fonts/truetype.nanum/NanumBarunGothic.ttf'

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 16 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 0s (24.8 MB/s)
Selecting previously unselected package fonts-nanum.
(Reading database ... 120831 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [ ]:
#부정적인 텍스트 가져오기 [빈칸10]


In [ ]:
#워드클라우드 시각화
plt.figure(figsize=(10,8))
plt.imshow(cloud)

In [ ]:
#긍정적인 텍스트 가져오기 [빈칸11]


In [ ]:
#워드클라우드 시각화
plt.figure(figsize=(10,8))
plt.imshow(cloud)